In [1]:
import pandas as pd
import altair as alt
import numpy as np
import wget

In [2]:
urls = [
     'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv',
     'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
 ]
[wget.download(url) for url in urls]

100% [..........................................................................] 8807778 / 8807778

['time_series_covid19_confirmed_US.csv', 'time_series_covid19_deaths_US.csv']

In [3]:
pd.set_option('display.max_columns', None)
# Load data
confirmed_usa_df = pd.read_csv('time_series_covid19_confirmed_US.csv')
deaths_usa_df = pd.read_csv('time_series_covid19_deaths_US.csv')

In [4]:
confirmed_usa_df.drop(confirmed_usa_df.columns[[0, 1, 2, 3, 4, 10]], axis = 1, inplace = True)
confirmed_dates = confirmed_usa_df.columns[5:]
confirmed_df_long = confirmed_usa_df.melt(
    id_vars=['Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_'],
    value_vars=confirmed_dates,
    var_name='Date',
    value_name='Confirmed'
)

In [5]:
deaths_usa_df.drop(deaths_usa_df.columns[[0, 1, 2, 3, 4, 10]], axis = 1, inplace = True)
deaths_dates = deaths_usa_df.columns[6:]
deaths_df_long = deaths_usa_df.melt(
    id_vars=['Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Population'],
    value_vars=deaths_dates,
    var_name='Date',
    value_name='Deaths'
)

In [6]:
confirmed_df_long['Date'] = pd.to_datetime(confirmed_df_long['Date'])
confirmed_df_long.loc[confirmed_df_long.Province_State.isna(),'Province_State'] = ''
confirmed_df_long = confirmed_df_long[(confirmed_df_long.Country_Region == 'US')].copy()

deaths_df_long['Date'] = pd.to_datetime(deaths_df_long['Date'])
deaths_df_long.loc[deaths_df_long.Province_State.isna(),'Province_State'] = ''
deaths_df_long = deaths_df_long[(deaths_df_long.Country_Region == 'US')].copy()

In [7]:
#hide
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
state2abbr = {s:a for a,s in states.items()}

def clean_state(col):
    if ',' in col:
        return col.split(', ', 1)[1]
    else:
        return state2abbr.get(col)


In [8]:

confirmed_df_long['state'] = confirmed_df_long.Province_State.apply(clean_state)

confirmed_df_long = confirmed_df_long.replace(to_replace='None', value=np.nan).dropna()

confirmed_df_long.drop(confirmed_df_long.columns[[ 1, 2, 3, 4]], axis = 1, inplace = True)

deaths_df_long['state'] = deaths_df_long.Province_State.apply(clean_state)

deaths_df_long = deaths_df_long.replace(to_replace='None', value=np.nan).dropna()

# print(deaths_df_long.head())
deaths_df_long.drop(deaths_df_long.columns[[ 1, 2, 3, 4]], axis = 1, inplace = True)


In [9]:

us_state_daily_confirmed_df = (confirmed_df_long
         .groupby(['state','Date'])
         .agg(
             Confirmed=('Confirmed','sum'))
        )

us_state_daily_death_df = (deaths_df_long
         .groupby(['state','Date'])
         .agg(
             Deaths=('Deaths','sum'),
            Population=('Population', 'sum')
            )
        )

In [10]:
us_state_daily_confirmed_df['new_cases'] = us_state_daily_confirmed_df.Confirmed.diff()
us_state_daily_confirmed_df.loc[us_state_daily_confirmed_df.new_cases < 0, 'new_cases'] = 0
us_state_daily_confirmed_df = us_state_daily_confirmed_df.reset_index()

us_state_daily_death_df['new_deaths'] = us_state_daily_death_df.Deaths.diff()
us_state_daily_death_df.loc[us_state_daily_death_df.new_deaths < 0, 'new_deaths'] = 0
us_state_daily_death_df = us_state_daily_death_df.reset_index()

In [11]:
usa_state_confirmed_death_df = us_state_daily_confirmed_df.merge(
   right=us_state_daily_death_df,
   how='left',
   on=['state', 'Date']
 )

usa_state_confirmed_death_df = usa_state_confirmed_death_df.fillna(0)
# fixing data types
cols = ['new_cases', 'Confirmed','Deaths', 'new_deaths', 'Population' ]
usa_state_confirmed_death_df[cols] = usa_state_confirmed_death_df[cols].astype('int')

usa_confirmed_death_df=(usa_state_confirmed_death_df
         .groupby(['Date'])
         .agg(
             Confirmed=('Confirmed','sum'),
             new_cases=('new_cases', 'sum'),
             Deaths=('Deaths', 'sum'),
             new_deaths=('new_deaths', 'sum'),
             Population=('Population', 'sum')
            )
        )
usa_confirmed_death_df = usa_confirmed_death_df.reset_index()

In [12]:

print(usa_state_confirmed_death_df.tail())
print(usa_confirmed_death_df.tail())

      state       Date  Confirmed  new_cases  Deaths  Population  new_deaths
45651    WY 2022-06-13     159907          0    1824      578759           0
45652    WY 2022-06-14     162160       2253    1824      578759           0
45653    WY 2022-06-15     162160          0    1824      578759           0
45654    WY 2022-06-16     162160          0    1824      578759           0
45655    WY 2022-06-17     162160          0    1824      578759           0
          Date  Confirmed  new_cases   Deaths  new_deaths  Population
873 2022-06-13   85578022     150586  1010991         276   332492855
874 2022-06-14   85669953      91931  1011372         381   332492855
875 2022-06-15   85852460     182507  1012052         683   332492855
876 2022-06-16   85968741     116281  1012092         420   332492855
877 2022-06-17   86127209     158468  1012802         710   332492855


In [13]:
base = alt.Chart(usa_confirmed_death_df).mark_bar().encode(
    x='yearmonth(Date):O',
).properties(
    width=500
)
red = alt.value("#f54242")
base.encode(y='Confirmed').properties(title='Total Confirmed')|base.encode(y='Deaths', color=red).properties(title='Total Deaths')


alt.HConcatChart(...)

In [14]:
base = alt.Chart(usa_confirmed_death_df).mark_bar().encode(
    x='yearmonth(Date):O',
).properties(
    width=500
)
red = alt.value("#f54242")
base.encode(y='new_cases').properties(title='Daily Cases')|base.encode(y='new_deaths', color=red).properties(title='Daily Deaths')


alt.HConcatChart(...)

In [15]:
top_impacted_states = usa_state_confirmed_death_df.nlargest(n=5, columns=['Date','Confirmed']).state.head(5).tolist()

print(top_impacted_states)

print(usa_state_confirmed_death_df.head())
top_five_state_df=usa_state_confirmed_death_df[usa_state_confirmed_death_df['state'].isin(top_impacted_states)]

print(top_five_state_df.head())

alt.Chart(top_five_state_df).mark_circle().encode(
    x='yearmonth(Date):O',
    y='state',
    color='state',
    size=alt.Size('new_cases:Q',
        scale=alt.Scale(range=[0, 1000]),
        legend=alt.Legend(title='Daily new cases')
    ) 
).properties(
    width=800,
    height=300
)


['CA', 'TX', 'FL', 'NY', 'IL']
  state       Date  Confirmed  new_cases  Deaths  Population  new_deaths
0    AK 2020-01-22          0          0       0      740995           0
1    AK 2020-01-23          0          0       0      740995           0
2    AK 2020-01-24          0          0       0      740995           0
3    AK 2020-01-25          0          0       0      740995           0
4    AK 2020-01-26          0          0       0      740995           0
     state       Date  Confirmed  new_cases  Deaths  Population  new_deaths
3512    CA 2020-01-22          0          0       0    39512223           0
3513    CA 2020-01-23          0          0       0    39512223           0
3514    CA 2020-01-24          0          0       0    39512223           0
3515    CA 2020-01-25          0          0       0    39512223           0
3516    CA 2020-01-26          2          2       0    39512223           0


alt.Chart(...)

In [16]:

alt.Chart(top_five_state_df).mark_line().encode(
    x='yearmonth(Date):O',
    color="state:N",
    y='new_cases:Q',
    tooltip=["state:N", "new_cases"]
)

alt.Chart(...)